In [1]:
import pandas as pd
import usaddress
import pyodbc
from fuzzywuzzy import fuzz
import useful_functions as use
from functools import reduce

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [45]:
#ppd
ppd = use.get_ppd()
ppd['IQVIA_ME'] = [x[0:10] for x in ppd.ME]

C:/Users/vigrose/Data/PPD\ppd_data_20210724.csv


In [6]:
#iqvia
username = 'vigrose'
password = 'Ravenclaw~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHONE,
        B.NUM_OF_PROVIDERS,
        B.PROFIT_STATUS,
        B.OWNER_STATUS,
        B.PHYSICAL_ADDR_1,
        B.PHYSICAL_ADDR_2,
        B.PHYSICAL_CITY,
        B.PHYSICAL_STATE,
        B.PHYSICAL_ZIP,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()
iqvia =iqvia.fillna('None')
iqvia['IQVIA_PHONE'] = [str(int(x)) if x !='None' else x for x in iqvia['PHONE']]
iqvia = iqvia.rename(columns={
    'PHYSICAL_ZIP':'ZIP_IQVIA',
    'PHYSICAL_STATE':'STATE_IQVIA',
    'PHYSICAL_CITY':'CITY_IQVIA',
    'PHYSICAL_ADDR_1': 'ADDRESS_1_IQVIA',
    'PHYSICAL_ADDR_2': 'ADDRESS_2_IQVIA'
})

In [17]:
iqvia.groupby('OWNER_STATUS').count()

,PHONE,NUM_OF_PROVIDERS,PROFIT_STATUS,ADDRESS_1_IQVIA,ADDRESS_2_IQVIA,CITY_IQVIA,STATE_IQVIA,ZIP_IQVIA,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK,IQVIA_PHONE
OWNER_STATUS,,,,,,,,,,,,,
ALLIED,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987
INDEPENDENT,79084,79084,79084,79084,79084,79084,79084,79084,79084,79084,79084,79084,79084
NOT INDEPENDENT,1466749,1466749,1466749,1466749,1466749,1466749,1466749,1466749,1466749,1466749,1466749,1466749,1466749


In [46]:
ppd['PE'] = [present_employment_key[x] for x in ppd.PE_CD]

In [47]:
ppd = ppd[['ME', 'MAILING_NAME',
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_CITY', 'POLO_STATE', 'POLO_ZIP', 'IQVIA_ME', 'PE', 'TOP_CD']]

In [23]:
iqvia['IQVIA_ZIP'] = [use.fix_zipcode(x) for x in iqvia.ZIP_IQVIA]

In [51]:
why = pd.merge(ppd, iqvia, left_on=['IQVIA_ME', 'POLO_ZIP'], right_on=['ME', 'IQVIA_ZIP'])[['POLO_MAILING_LINE_1','POLO_MAILING_LINE_2','ADDRESS_1_IQVIA', 'PE','NUM_OF_PROVIDERS','PROFIT_STATUS', 'OWNER_STATUS', 'TOP_CD']]

In [52]:
why = why.fillna('None')

In [53]:
why.groupby('TOP_CD').count()

,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,ADDRESS_1_IQVIA,PE,NUM_OF_PROVIDERS,PROFIT_STATUS,OWNER_STATUS
TOP_CD,,,,,,,
12,11072,11072,11072,11072,11072,11072,11072
20,130741,130741,130741,130741,130741,130741,130741
30,1506,1506,1506,1506,1506,1506,1506
40,2395,2395,2395,2395,2395,2395,2395
50,2501,2501,2501,2501,2501,2501,2501
62,374,374,374,374,374,374,374
71,4770,4770,4770,4770,4770,4770,4770
72,1490,1490,1490,1490,1490,1490,1490
74,71,71,71,71,71,71,71


In [54]:
pe_dict = {}
for row in why[why.TOP_CD==20].itertuples():
    if is_a_match(row.POLO_MAILING_LINE_2, row.ADDRESS_1_IQVIA):
        if row.PE in pe_dict.keys():
            pe_dict[row.PE]['NUMBERS'].append(row.NUM_OF_PROVIDERS)
            pe_dict[row.PE]['PROFIT_STATUS'].append(row.PROFIT_STATUS)
            pe_dict[row.PE]['OWNER_STATUS'].append(row.OWNER_STATUS)
        else:
            pe_dict[row.PE] = {}
            pe_dict[row.PE]['NUMBERS'] = [row.NUM_OF_PROVIDERS]
            pe_dict[row.PE]['PROFIT_STATUS'] = [row.PROFIT_STATUS]
            pe_dict[row.PE]['OWNER_STATUS'] = [row.OWNER_STATUS]

In [61]:
listy = []
for kee in pe_dict.keys():
    NEW_DICT = {}
    NEW_DICT['PE'] = kee
    this = pe_dict[kee]['PROFIT_STATUS']
    this_2 = pe_dict[kee]['OWNER_STATUS']
    nums = pe_dict[kee]['NUMBERS']
    NEW_DICT['PROFIT_STATUS'] = max(set(this), key = this.count)
    NEW_DICT['OWNER_STATUS'] = max(set(this_2), key = this.count)
    NEW_DICT['NUMBER_OF_PROVIDERS_MEAN'] = sum(nums)/len(nums)
    NEW_DICT['NUMBER_OF_PROVIDERS_MODE'] = max(set(nums), key = this.count)
    NEW_DICT['COUNT'] = len(nums)
    listy.append(NEW_DICT)

In [60]:
why.groupby(['PE','PROFIT_STATUS']).count()

POLO_MAILING_LINE_1  \
PE                                               PROFIT_STATUS                         
City/County/State Government-Hospital            For Profit                     1505   
                                                 Government                     2715   
                                                 None                             65   
                                                 Not for Profit                17256   
City/County/State Government-Other Than Hospital For Profit                       60   
...                                                                              ...   
Self-Employed Solo Practice                      Not for Profit                 8967   
Two Physician Practice-Full Or Part Owner        For Profit                      927   
                                                 Government                      145   
                                                 None                             19   
                                                 Not for Profit                 2270   

                                                                 POLO_MAILING_LINE_2  \
PE                                               PROFIT_STATUS                         
City/County/State Government-Hospital            For Profit                     1505   
                                                 Government                     2715   
                                                 None                             65   
                                                 Not for Profit                17256   
City/County/State Government-Other Than Hospital For Profit                       60   
...                                                                              ...   
Self-Employed Solo Practice                      Not for Profit                 8967   
Two Physician Practice-Full Or Part Owner        For Profit                      927   
                                                 Government                      145   
                                                 None                             19   
                                                 Not for Profit                 2270   

                                                                 ADDRESS_1_IQVIA  \
PE                                               PROFIT_STATUS                     
City/County/State Government-Hospital            For Profit                 1505   
                                                 Government                 2715   
                                                 None                         65   
                                                 Not for Profit            17256   
City/County/State Government-Other Than Hospital For Profit                   60   
...                                                                          ...   
Self-Employed Solo Practice                      Not for Profit             8967   
Two Physician Practice-Full Or Part Owner        For Profit                  927   
                                                 Government                  145   
                                                 None                         19   
                                                 Not for Profit             2270   

                                                                 NUM_OF_PROVIDERS  \
PE                                               PROFIT_STATUS                      
City/County/State Government-Hospital            For Profit                  1505   
                                                 Government                  2715   
                                                 None                          65   
                                                 Not for Profit             17256   
City/County/State Government-Other Than Hospital For Profit                    60   
...                                                                           ...   
Se

In [62]:
pd.DataFrame(listy)

,PE,PROFIT_STATUS,OWNER_STATUS,NUMBER_OF_PROVIDERS_MEAN,NUMBER_OF_PROVIDERS_MODE,COUNT
0,Other-Patient Care,Not for Profit,ALLIED,605.620779,1,385
1,City/County/State Government-Hospital,Not for Profit,ALLIED,994.473892,1,13444
2,Non-Government Hospital,Not for Profit,ALLIED,941.799129,1,7577
3,Group Practice,Not for Profit,ALLIED,766.359941,1,22437
4,Medical School,Not for Profit,ALLIED,1498.052850,3,965
5,Two Physician Practice-Full Or Part Owner,Not for Profit,ALLIED,420.186007,1,586
6,No Classification,Not for Profit,ALLIED,1040.928772,1,14938
7,Self-Employed Solo Practice,Not for Profit,ALLIED,516.713775,1,2519
8,Federal Government-Hospital/Navy,Government,NOT INDEPENDENT,637.691011,1,178
9,Federal Government-Hospital/Vet Admin,Government,NOT INDEPENDENT,440.617363,1536,1244


In [11]:
present_employment_key = {
        11: 'Self-Employed Solo Practice',
        13: 'Two Physician Practice-Full Or Part Owner',
        21: 'Other-Patient Care',
        22: 'Locum Tenens',
        30: 'Group Practice',
        35: 'HMO',
        40: 'Medical School',
        50: 'Non-Government Hospital',
        63: 'City/County/State Government-Hospital',
        64: 'City/County/State Government-Other Than Hospital',
        81: 'Federal Government-Hospital/Army',
        82: 'Federal Government-Hospital/Navy',
        83: 'Federal Government-Hospital/Air Force',
        84: 'Federal Government-Hospital/Usphs',
        85: 'Federal Government-Hospital/Vet Admin',
        86: 'Federal Government-Hospital/Other Agency',
        101: 'Other/Non-Patient Care',
        110: 'No Classification'
    }

In [ ]:
THIS = ppd[['ME','IQVIA_ME','TELEPHONE_NUMBER']]
THIS = pd.merge(THIS, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')
THIS = pd.merge(THIS, gov, on='ME', how='left')[['ME','IQVIA_ME', 'TELEPHONE_NUMBER','SYM_PHONE','IQVIA_PHONE', 'DHC_PHONE', 'GOV_PHONE']].drop_duplicates()
THIS['TELEPHONE_NUMBER'] = [use.fix_phone(x) for x in THIS.TELEPHONE_NUMBER]
THIS = THIS.fillna('None')

In [ ]:
zip_matches.mean()

In [ ]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    MATCHES = ''
    phone = row.TELEPHONE_NUMBER
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    data_phone = row.GOV_PHONE
    if phone.isnumeric():
        if symph_phone == phone:
            count += 1
            MATCHES += 'Symphony, '
        if dhc_phone == phone:
            count += 1
            MATCHES += 'DHC, '
        if iqvia_phone == phone:
            count += 1
            MATCHES += 'IQVia, '
        if data_phone == phone:
            count += 1
            MATCHES += 'DataGov, '
        dicto = {
            'ME': row.ME,
            'IQVIA_ME': row.IQVIA_ME,
            'PHONE_PPD': row.TELEPHONE_NUMBER,
            'MATCHED': count,
            'MATCHES': MATCHES[:-2]
        }
        dict_list.append(dicto)

In [ ]:
phone_matches = pd.DataFrame(dict_list)

In [ ]:
phone_matches = phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates('ME')

In [ ]:
phone_matches.groupby('MATCHED').count()[['ME']]

In [11]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [12]:
THIS = dpc[['ME','IQVIA_ME','POLO_MAILING_LINE_2']]
THIS = pd.merge(THIS, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')
THIS = pd.merge(THIS, gov, on='ME', how='left')[['ME','IQVIA_ME', 'POLO_MAILING_LINE_2','ADDRESS_1_SYMPHONY','ADDRESS_1_IQVIA', 'ADDRESS_1_DHC', 'ADDRESS_1_GOV']].drop_duplicates()
THIS = THIS.fillna('None')

In [14]:
THIS

,ME,IQVIA_ME,POLO_MAILING_LINE_2,ADDRESS_1_SYMPHONY,ADDRESS_1_IQVIA,ADDRESS_1_DHC,ADDRESS_1_GOV
0,00102000013,0010200001,None,4899 REYNOLDS LN,1613 N MCKENZIE ST,440 Taylor Rd,None
1,00102000013,0010200001,None,4899 REYNOLDS LN,201 14TH ST SW,440 Taylor Rd,None
2,00102000030,0010200003,500 CENTREPARK DR,500 CENTRE PARK DR,428 BILTMORE AVE,500 Centre Park Dr,None
3,00102000030,0010200003,500 CENTREPARK DR,500 CENTRE PARK DR,509 BILTMORE AVE,500 Centre Park Dr,None
4,00102000048,0010200004,14502 W MEEKER BLVD,14502 W MEEKER BLVD,14502 W MEEKER BLVD,14502 W Meeker Blvd,1111 E MCDOWELL RD
...,...,...,...,...,...,...,...
2958525,96501960044,9650196004,142 S MAIN ST,142 S MAIN ST,3000 NEW BERN AVE,142 S Main St,144 E FERRELL ST
2958531,96501960052,9650196005,1 MEDICAL CENTER BLVD,1 MEDICAL CENTER BLVD,MEDICAL CENTER BLVD,Medical Center Blvd,250 HOSPITAL DR
2958532,96501960052,9650196005,1 MEDICAL CENTER BLVD,1 MEDICAL CENTER BLVD,MEDICAL CENTER BLVD,Medical Center Blvd,MEDICAL CTR BLVD
2958533,96501960052,9650196005,1 MEDICAL CENTER BLVD,1 MEDICAL CENTER BLVD,1 MEDICAL CENTER BLVD,Medical Center Blvd,250 HOSPITAL DR


In [27]:
def is_a_match(thing_1, thing_2):
    thing_1 = thing_1.strip().upper()
    thing_2 = thing_2.strip().upper()
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>80:
        match = True
    else:
        match = False
    return(match)

In [18]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    total = 4 - row.count('None')
    MATCHES = ''
    phone = row.POLO_MAILING_LINE_2
    symph_phone = row.ADDRESS_1_SYMPHONY
    dhc_phone =row.ADDRESS_1_DHC
    iqvia_phone = row.ADDRESS_1_IQVIA
    data_phone = row.ADDRESS_1_GOV
    if phone!='None':
        if is_a_match(symph_phone,phone):  
            count += 1
            MATCHES += 'Symphony, '
        if is_a_match(dhc_phone,phone):
            count += 1
            MATCHES += 'DHC, '
        if is_a_match(iqvia_phone,phone):
            count += 1
            MATCHES += 'IQVia, '
        if is_a_match(data_phone,phone):
            count += 1
            MATCHES += 'DataGov, '
        dicto = {
            'ME': row.ME,
            'IQVIA_ME': row.IQVIA_ME,
            'ADDRESS_PPD': phone,
            'MATCHED': count,
            'MATCHES': MATCHES[:-2],
            'COVERAGE': total
        }
        dict_list.append(dicto)

In [19]:
polo_matches = pd.DataFrame(dict_list).sort_values('MATCHED', ascending=False).drop_duplicates('ME')

In [20]:
polo_matches

,ME,IQVIA_ME,ADDRESS_PPD,MATCHED,MATCHES,COVERAGE
1140089,03608010245,0360801024,7777 FOREST LN STE A337,4,"Symphony, DHC, IQVia, DataGov",4
751664,02604862933,0260486293,1200 6TH AVE N,4,"Symphony, DHC, IQVia, DataGov",4
752409,02604873161,0260487316,701 PARK AVE,4,"Symphony, DHC, IQVia, DataGov",4
752406,02604873129,0260487312,725 AMERICAN AVE,4,"Symphony, DHC, IQVia, DataGov",4
752217,02604871959,0260487195,701 PARK AVE,4,"Symphony, DHC, IQVia, DataGov",4
...,...,...,...,...,...,...
339981,01611101968,0161110196,210 SAINT JOSEPH DR FL 3,0,,4
339980,01611101950,0161110195,2822 E 83RD ST,0,,2
339979,01611101933,0161110193,675 W KIRCHHOFF RD,0,,4
339971,01611101861,0161110186,55 LAKE AVE N,0,,3


In [ ]:
dict_list = []
sym_count = 0
dhc_count = 0
iqvia_count = 0
data_count = 0
for row in THIS.itertuples():
    count = 0
    total = 4 - row.count('None')
    MATCHES = ''
    phone = use.fix_zipcode(row.POLO_ZIP)
    symph_phone = use.fix_zipcode(row.ZIP_SYMPHONY)
    dhc_phone =use.fix_zipcode(row.ZIP_DHC)
    iqvia_phone = use.fix_zipcode(row.ZIP_IQVIA)
    data_phone = use.fix_zipcode(row.ZIP_GOV)
    if phone.isnumeric():
        if symph_phone == phone:
            sym_count+=1
            count += 1
            MATCHES += 'Symphony, '
        if dhc_phone == phone:
            dhc_count+=1
            count += 1
            MATCHES += 'DHC, '
        if iqvia_phone == phone:
            iqvia_count+=1
            count += 1
            MATCHES += 'IQVia, '
        if data_phone == phone:
            data_count+=1
            count += 1
            MATCHES += 'DataGov, '
        dicto = {
            'ME': row.ME,
            'IQVIA_ME': row.IQVIA_ME,
            'ZIP_PPD': row.POLO_ZIP,
            'MATCHED': count,
            'MATCHES': MATCHES[:-2],
            'COVERAGE': total
        }
        dict_list.append(dicto)

In [ ]:
zip_matches = pd.DataFrame(dict_list).sort_values('MATCHED', ascending=False).drop_duplicates('ME')

In [24]:
polo_matches.groupby('PERCENT').count()[['ME']]

,ME
PERCENT,
0.000000,143122
0.250000,56182
0.333333,57825
0.500000,112046
0.666667,80550
0.750000,114872
1.000000,155276
2.000000,1
3.000000,3


In [23]:
polo_matches['PERCENT'] = polo_matches["MATCHED"] / polo_matches["COVERAGE"]

In [ ]:
zip_matches.groupby('PERCENT').count()[['ME']]

In [ ]:
print(f'''
{sym_count}
{dhc_count}
{iqvia_count}
{data_count}
''')

In [ ]:
THIS[(THIS.POLO_ZIP=='None')].drop_duplicates('ME')

In [25]:
XX = dpc[['ME','PE_CD','PRIM_SPEC_CD']]

In [26]:
pd.merge(polo_matches, XX, on='ME').groupby('PE_CD').mean().to_csv('../../Data/Gap_Analysis/PE_CD_POLO_MATCH.csv')

In [28]:
SPEC = pd.read_csv('../../Data/PPD/speciality_id.csv')
prim_spec = pd.merge(polo_matches, XX, on='ME').groupby('PRIM_SPEC_CD').mean()
pd.merge(prim_spec, SPEC, left_on='PRIM_SPEC_CD',right_on='SPEC_CD', how='left').to_csv('../../Data/Gap_Analysis/SPEC_POLO_MATCH.csv')

In [ ]:
len(dpc)

In [ ]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    phone_num = 'None'
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    data_phone = row.GOV_PHONE
    print(iqvia_phone)
    if symph_phone == dhc_phone and symph_phone != 'None':
        MATCHES = 'Symphony, DHC'
        phone_num = symph_phone
        if dhc_phone == iqvia_phone:
            count = 2
            MATCHES = 'Symphony, DHC, IQVia'
            if dhc_phone == data_phone:
                count = 3
                MATCHES = 'Symphony, DHC, IQVia, DataGov'
        elif dhc_phone == data_phone:
            count = 2
            MATCHES = 'Symphony, DHC, DataGov'
        else:
            count = 1       
    elif symph_phone == iqvia_phone and symph_phone != 'None':
        phone_num = symph_phone
        MATCHES = 'Symphony, IQVia'
        count = 1
        if symph_phone == data_phone:
            count = 2
            MATCHES = 'Symphony, IQVia, DataGov'
    elif symph_phone == data_phone and symph_phone != 'None':
        phone_num = symph_phone
        MATCHES = 'Symphony, DataGov'
        count = 1
        
    elif dhc_phone == iqvia_phone and dhc_phone != 'None':
        phone_num = dhc_phone
        MATCHES = 'DHC, IQVia'
        count = 1
        if dhc_phone == data_phone:
            MATCHES = 'DHC, IQVia, DataGov'
            count = 2
    elif dhc_phone == data_phone and dhc_phone != 'None':
        phone_num = dhc_phone
        MATCHES = 'DHC, DataGov'
        count = 1
        
    if phone_num!='None':
        dicto = {
        'ME': row.ME,
        'IQVIA_ME': row.IQVIA_ME,
        'PHONE': phone_num,
        'MATCHED': count,
        'MATCHES': MATCHES
    }
        dict_list.append(dicto)

In [ ]:
phone_matches = pd.DataFrame(dict_list)

In [ ]:
best = phone_matches[phone_matches.MATCHED==3]

In [ ]:
phone_matches.drop_duplicates('ME')

In [ ]:
def clean_address_two(add_1):
    add_1 = add_1.strip()
    if add_1 == 'None':
        addr_1 = ' '
    elif add_1 == 'NAN':
        addr_1 = ' '
    else:
        addr_1 = ',' + add_1
    return(addr_1)

def is_a_match(thing_1, thing_2):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>70:
        match = True
    else:
        match = False
    return(match)

def is_match(thing_1, thing_2):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    else:
        match = False
    return(match)

def error_handle(parsed_string):
    new_dict = {}
    for thing in parsed_string:
        if thing[1] in new_dict.keys():
            a_list = [new_dict[thing[1]], thing[0]]
            new_dict[thing[1]] = max(a_list, key=len)
        else:
            new_dict[thing[1]] = thing[0]
    return(new_dict)

# def get_all_keys(moar):
#     all_keys =[]
#     for row in moar.itertuples():
#         addr_2 = clean_address_two(row.ADDRESS_2)
#         address = f'{row.ADDRESS_1}{addr_2}, {row.CITY}, {row.STATE}'
#         try:
#             new_dict = usaddress.tag(address)[0]
#         except usaddress.RepeatedLabelError as e:
#             print(e.original_string)
#             new_dict = error_handle(e.parsed_string)
#             print('')
#         le_keys = list(new_dict.keys())
#         for key in le_keys:
#             if key not in all_keys:
#                 all_keys.append(key)
#         return (all_keys)
    
# def parse_address(moar):
#     dict_list = []
#     for row in moar.itertuples():
#         new_dict = {}
#         new_dict['ME'] = row.ME
#         new_dict['IQVIA_ME'] = row.IQVIA_ME
#         new_dict['PHONE'] = row.PHONE
#         new_dict['ZIP'] = row.ZIP
#         addr_2 = clean_address_two(row.ADDRESS_2)
#         address = f'{row.ADDRESS_1}{addr_2}, {row.CITY}, {row.STATE}'
#         try:
#             address_dict = usaddress.tag(address)[0]
#         except usaddress.RepeatedLabelError as e:
#             print(e.original_string)
#             address_dict = error_handle(e.parsed_string)
#             print('')
#         dict_list.append(new_dict)
#     return (dict_list)

# LE_KEYS = get_all_keys(all_file)
# new_list = parse_address(all_file, LE_KEYS)

In [ ]:
def count_matches(THIS):
    dict_list = []
    for row in THIS.itertuples():
        count = 0
        phone_num = 'None'
        symph_phone = row.ADDRESS_1_SYMPHONY.upper().strip()
        dhc_phone = row.ADDRESS_1_DHC.upper().strip()
        iqvia_phone = row.ADDRESS_1_IQVIA.upper().strip()
        data_phone = row.ADDRESS_1_GOV.upper().strip()
        print(iqvia_phone)
        if symph_phone == dhc_phone and symph_phone != 'NONE':
            MATCHES = 'Symphony, DHC'
            phone_num = symph_phone
            if dhc_phone == iqvia_phone:
                count = 2
                MATCHES = 'Symphony, DHC, IQVia'
                if dhc_phone == data_phone:
                    count = 3
                    MATCHES = 'Symphony, DHC, IQVia, DataGov'
            elif dhc_phone == data_phone:
                count = 2
                MATCHES = 'Symphony, DHC, DataGov'
            else:
                count = 1       
        elif symph_phone == iqvia_phone and symph_phone != 'NONE':
            phone_num = symph_phone
            MATCHES = 'Symphony, IQVia'
            count = 1
            if symph_phone == data_phone:
                count = 2
                MATCHES = 'Symphony, IQVia, DataGov'
        elif symph_phone == data_phone and symph_phone != 'NONE':
            phone_num = symph_phone
            MATCHES = 'Symphony, DataGov'
            count = 1

        elif dhc_phone == iqvia_phone and dhc_phone != 'NONE':
            phone_num = dhc_phone
            MATCHES = 'DHC, IQVia'
            count = 1
            if dhc_phone == data_phone:
                MATCHES = 'DHC, IQVia, DataGov'
                count = 2
        elif dhc_phone == data_phone and dhc_phone != 'NONE':
            phone_num = dhc_phone
            MATCHES = 'DHC, DataGov'
            count = 1

        if phone_num!='None':
            dicto = {
            'ME': row.ME,
            'IQVIA_ME': row.IQVIA_ME,
            'ADDRESS': phone_num,
            'MATCHED': count,
            'ADDRESS_MATCHES': MATCHES
        }
            dict_list.append(dicto)
    return dict_list

In [ ]:
this = older_polos[['ME','IQVIA_ME']]
this = pd.merge(this, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
this = pd.merge(this, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
this = pd.merge(this, dhc, on='ME', how='left')
this = pd.merge(this, gov, on='ME', how='left').drop_duplicates()

In [ ]:
this = this.fillna('None')
ADD_LIST = count_matches(this)

In [ ]:
ADDRESSES = pd.DataFrame(ADD_LIST).drop_duplicates(['ME','ADDRESS'])

In [ ]:
alls = pd.merge(ADDRESSES, phone_matches, on=['ME','IQVIA_ME'], how='outer', suffixes=['_ADDRESS','_PHONE']).drop_duplicates()

In [ ]:
this

In [ ]:
alls.sort_values('MATCHED_PHONE', ascending=False)

In [ ]:
TEES = pd.merge(alls, THIS, on=['ME','IQVIA_ME']).sort_values('MATCHED_PHONE', ascending=False)

In [ ]:
XX = pd.merge(TEES, this, on=['ME','IQVIA_ME']).sort_values('MATCHED_PHONE', ascending=False)

In [ ]:
fuzzed_xx = pd.merge(fuzzed_cont, this, on=['ME','IQVIA_ME'])

In [ ]:
this[['ME', 'IQVIA_ME', 'ADDRESS_2_SYMPHONY', 'ZIP_SYMPHONY',  'ADDRESS_1_IQVIA', 
       'ZIP_IQVIA', 'ZIP_DHC', 'ADDRESS_1_DHC', 'First Name', 'Physician Name',
       'Middle Name', 'Primary Specialty', 'Primary Hospital Affiliation',
       'Last Name', 'ADDRESS_2_DHC', 'DHC_PHONE', 'NPI_y', ' Ind_PAC_ID',
       ' Ind_enrl_ID', ' lst_nm', ' frst_nm', ' mid_nm', ' suff', ' gndr',
       ' Cred', ' Med_sch', ' Grd_yr', ' pri_spec', ' sec_spec_1',
       ' sec_spec_2', ' sec_spec_3', ' sec_spec_4', ' sec_spec_all', ' org_nm',
       ' org_pac_id', ' num_org_mem', 'ADDRESS_1_GOV', 'ADDRESS_2_GOV',
       ' ln_2_sprs', 'CITY_GOV', 'STATE_GOV', 'ZIP_GOV', ' phn_numbr',
       ' hosp_afl_1', ' hosp_afl_lbn_1', ' hosp_afl_2', ' hosp_afl_lbn_2',
       ' hosp_afl_3', ' hosp_afl_lbn_3', ' hosp_afl_4', ' hosp_afl_lbn_4',
       ' hosp_afl_5', ' hosp_afl_lbn_5', ' ind_assgn', ' grp_assgn',
       ' adrs_id', 'PARTY_ID', 'GOV_PHONE']]

In [ ]:
iqvia[iqvia.ME.isin(TEES.IQVIA_ME)==False].drop_duplicates('ME')

In [ ]:
TEES.drop_duplicates('ME')

In [ ]:
phone_matches.drop_duplicates('ME')

In [ ]:
8161+5294

In [ ]:
pd.merge(testt, older_polos, on=['ME','IQVIA_ME'])[['ADDRESS','STATE_DHC','STATE_IQVIA','STATE_SYMPHONY','STATE_GOV','STATE_POLO','STATE_PPMA','ADDRESS_MATCHES']].drop_duplicates()

In [ ]:
older_polos.columns

In [ ]:
len(older_polos)

In [ ]:
testt = XX[['ME', 'IQVIA_ME', 'ADDRESS', 'MATCHED_ADDRESS', 'ADDRESS_MATCHES',
       'PHONE_x', 'MATCHED_PHONE', 'MATCHES', 'STATE_SYMPHONY', 'STATE_IQVIA', 'STATE_DHC',
       'STATE_GOV']].drop_duplicates()

In [ ]:
testt

In [ ]:
XX.columns

In [ ]:
XX = XX.fillna('None')

In [ ]:
XX.drop_duplicates()

In [ ]:
dictss = []
mes = []
for row in XX.itertuples():
    if row.ADDRESS_MATCHES == 'None':
        print('Addres mismatch')
        continue
    if row.ME in mes:
        print('already found')
        continue
    sources = row.ADDRESS_MATCHES
    address = row.ADDRESS.upper().strip()
    phone = row.PHONE_x
    phone_source = 'Multiple'
    address_source = sources
    if "IQVia" in sources and row.ADDRESS_1_IQVIA.upper().strip() == address:
        state = row.STATE_IQVIA
        zipcode = row.ZIP_IQVIA
        city = row.CITY_IQVIA
        address_2 = row.ADDRESS_2_SYMPHONY
        if phone == 'None':
            phone = row.IQVIA_PHONE_x
            phone_source = 'IQVia'
        address_source = 'IQVia'
    elif "Symphony" in sources and row.ADDRESS_1_SYMPHONY.upper().strip() == address:
        state = row.STATE_SYMPHONY
        zipcode = row.ZIP_SYMPHONY
        city = row.CITY_SYMPHONY
        address_2 = row.ADDRESS_2_SYMPHONY
        if phone == 'None':
            phone = row.SYM_PHONE_x
            phone_source = 'Symphony'
        address_source = 'Symphony'
    elif "DataGov" in sources and row.ADDRESS_1_GOV.upper().strip() == address:
        state = row.STATE_GOV
        zipcode = row.ZIP_GOV
        city = row.CITY_GOV
        address_2 = row.ADDRESS_2_GOV
        if phone == 'None':
            phone = row.GOV_PHONE_x
            phone_source = 'DataGov'
        address_source = 'DataGov'
    NEW_DICT = {
        'ME': row.ME,
        'ME_IQVIA': row.IQVIA_ME,
        'ADDRESS_MATCHES': row.ADDRESS_MATCHES,
        'PHONE_MATCHES': row.MATCHES,
        'ADDRESS_1': address,
        'ADDRESS_2': address_2,
        'CITY': city,
        'STATE': state,
        'ZIPCODE': zipcode,
        'PHONE': phone,
        'PHONE_SOURCE': phone_source,
        'ADDRESS_SOURCE': address_source
    }
    dictss.append(NEW_DICT)
    if phone!='None':
        mes.append(row.ME)

In [ ]:
sample = pd.DataFrame(dictss).drop_duplicates(['ME','ADDRESS_1'])

In [ ]:
sample

In [ ]:
XX[XX.ADDRESS_MATCHES == 'None'][['ME','MATCHES',  'ADDRESS_1_SYMPHONY', 'CITY_SYMPHONY', 'STATE_SYMPHONY', 'ADDRESS_1_IQVIA',
       'CITY_IQVIA', 'STATE_IQVIA', 'STATE_DHC',
       'CITY_DHC', 'ADDRESS_1_DHC', 'ADDRESS_1_GOV', 'CITY_GOV', 'STATE_GOV']]

In [ ]:
no_address = XX[XX.ADDRESS_MATCHES == 'None']

In [ ]:
def count_fuzzy_matches(THIS):
    dict_list = []
    for row in THIS.itertuples():
        count = 0
        phone_num = 'None'
        symph_phone = row.ADDRESS_1_SYMPHONY.upper().strip()
        dhc_phone = row.ADDRESS_1_DHC.upper().strip()
        iqvia_phone = row.ADDRESS_1_IQVIA.upper().strip()
        data_phone = row.ADDRESS_1_GOV.upper().strip()
        print(iqvia_phone)
        if is_a_match(symph_phone,dhc_phone) and symph_phone != 'NONE':
            MATCHES = 'Symphony, DHC'
            phone_num = symph_phone
            if is_a_match(dhc_phone,iqvia_phone):
                count = 2
                MATCHES = 'Symphony, DHC, IQVia'
                if is_a_match(dhc_phone,data_phone):
                    count = 3
                    MATCHES = 'Symphony, DHC, IQVia, DataGov'
            elif is_a_match(dhc_phone,data_phone):
                count = 2
                MATCHES = 'Symphony, DHC, DataGov'
            else:
                count = 1       
        elif is_a_match(symph_phone,iqvia_phone) and symph_phone != 'NONE':
            phone_num = symph_phone
            MATCHES = 'Symphony, IQVia'
            count = 1
            if is_a_match(symph_phone,data_phone):
                count = 2
                MATCHES = 'Symphony, IQVia, DataGov'
        elif is_a_match(symph_phone,data_phone) and symph_phone != 'NONE':
            phone_num = symph_phone
            MATCHES = 'Symphony, DataGov'
            count = 1

        elif is_a_match(dhc_phone,iqvia_phone) and dhc_phone != 'NONE':
            phone_num = iqvia_phone
            MATCHES = 'DHC, IQVia'
            count = 1
            if is_a_match(iqvia_phone,data_phone):
                MATCHES = 'DHC, IQVia, DataGov'
                count = 2
        elif is_a_match(dhc_phone,data_phone) and dhc_phone != 'NONE':
            phone_num = data_phone
            MATCHES = 'DHC, DataGov'
            count = 1

        if phone_num!='None':
            dicto = {
            'ME': row.ME,
            'IQVIA_ME': row.IQVIA_ME,
            'ADDRESS': phone_num,
            'DHC_ADDRESS': dhc_phone,
            'SYM_ADDRESS': symph_phone,
            'IQV_ADDRESS': iqvia_phone,
            'GOV_ADDRESS': data_phone,
            'MATCHED': count,
            'ADDRESS_MATCHES': MATCHES
        }
            dict_list.append(dicto)
    return dict_list

In [ ]:
boom = count_fuzzy_matches(no_address)

In [ ]:
pd.DataFrame(boom).sort_values('MATCHED').drop_duplicates('ME', keep='last').to_csv("../../Data/POLO_Filter/Match_Test.csv", index=False)

In [ ]:
FUZZED = pd.DataFrame(boom).sort_values('MATCHED').drop_duplicates('ME', keep='last')

In [ ]:
THIS

In [ ]:
fuzzed_cont = pd.merge(FUZZED, phone_matches, on=['ME','IQVIA_ME'], suffixes=['_ADDRESSES','_PHONES']).drop_duplicates()

In [ ]:
#no phones?
#still no phones
#phones match but addresses do not
#100 mile check
#license check

In [ ]:
dicts_2 = []
mes_2 = []
for row in fuzzed_xx.itertuples():
    if row.ADDRESS_MATCHES == 'None':
        print('Addres mismatch')
        continue
    if row.ME in mes:
        print('already found')
        continue
    sources = row.ADDRESS_MATCHES
    address = row.ADDRESS.upper().strip()
    phone = row.PHONE_x
    phone_source = 'Multiple'
    address_source = sources
    if "IQVia" in sources and row.ADDRESS_1_IQVIA.upper().strip() == address:
        state = row.STATE_IQVIA
        zipcode = row.ZIP_IQVIA
        city = row.CITY_IQVIA
        address_2 = row.ADDRESS_2_SYMPHONY
        if phone == 'None':
            phone = row.IQVIA_PHONE_x
            phone_source = 'IQVia'
        address_source = 'IQVia'
    elif "Symphony" in sources and row.ADDRESS_1_SYMPHONY.upper().strip() == address:
        state = row.STATE_SYMPHONY
        zipcode = row.ZIP_SYMPHONY
        city = row.CITY_SYMPHONY
        address_2 = row.ADDRESS_2_SYMPHONY
        if phone == 'None':
            phone = row.SYM_PHONE_x
            phone_source = 'Symphony'
        address_source = 'Symphony'
    elif "DataGov" in sources and row.ADDRESS_1_GOV.upper().strip() == address:
        state = row.STATE_GOV
        zipcode = row.ZIP_GOV
        city = row.CITY_GOV
        address_2 = row.ADDRESS_2_GOV
        if phone == 'None':
            phone = row.GOV_PHONE_x
            phone_source = 'DataGov'
        address_source = 'DataGov'
    NEW_DICT = {
        'ME': row.ME,
        'ME_IQVIA': row.IQVIA_ME,
        'ADDRESS_MATCHES': row.ADDRESS_MATCHES,
        'PHONE_MATCHES': row.MATCHES,
        'ADDRESS_1': address,
        'ADDRESS_2': address_2,
        'CITY': city,
        'STATE': state,
        'ZIPCODE': zipcode,
        'PHONE': phone,
        'PHONE_SOURCE': phone_source,
        'ADDRESS_SOURCE': address_source
    }
    dicts_2.append(NEW_DICT)
    if phone!='None':
        mes_2.append(row.ME)

In [ ]:
pd.DataFrame(dicts_2).drop_duplicates()

In [ ]:
dhc

In [ ]:
dfs = [dhc, gov, older_polos]
df_final = reduce(lambda left,right: pd.merge(left,right,on='ME'), dfs)

In [ ]:
df1.merge(df2,on='name').merge(df3,on='name')

df_final

In [ ]:
THIS = older_polos[['ME','IQVIA_ME']]
THIS = pd.merge(THIS, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')
THIS = pd.merge(THIS, gov, on='ME', how='left')[['ME','IQVIA_ME', 'SYM_PHONE','IQVIA_PHONE', 'DHC_PHONE', 'GOV_PHONE']].drop_duplicates()

In [ ]:
dhc.columns

In [ ]:
def universalize_columns(df, source):
    df['ADDRESS_1'] = df[f'ADDRESS_1_{source}']
    df['ADDRESS_2'] = df[f'ADDRESS_2_{source}']
    df['CITY'] = df[f'CITY_{source}']
    df['STATE'] = df[f'STATE_{source}']
    df['ZIPCODE'] = df[f'ZIP_{source}']
    df['PHONE'] = df[f'{source}_PHONE']
    numb_list = list(range(0,len(df)))
    df['KEY'] = [str(s) + source for s in numb_list]

In [ ]:
source_list = [
    {'SOURCE':'DHC',
    'DATA':dhc},
    {'SOURCE':'IQVIA',
    'DATA':iqvia},
    {'SOURCE':'SYMPHONY',
    'DATA':symphony},
    {'SOURCE':'GOV',
    'DATA':gov}
]

In [ ]:
symphony['SYMPHONY_PHONE']=symphony.SYM_PHONE
symphony['IQVIA_ME'] = symphony.SYM_ME
iqvia['IQVIA_ME'] = iqvia.ME
dhc['IQVIA_ME'] = [x[0:10] for x in fix_me(dhc.ME)]
gov['IQVIA_ME'] = [x[0:10] for x in fix_me(gov.ME)]

In [ ]:
for source_dict in source_list:
    universalize_columns(source_dict['DATA'], source_dict['SOURCE'])

In [ ]:
def fix(component):
    component = component.strip().upper()
    return component

In [ ]:
def fix_zipcode(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    if len(num) > 5:
        num = num[:-4]
    if len(num) == 4:
        num = '0' + num
    elif len(num) == 3:
        num = '00' + num
    elif len(num) == 2:
        num = '000' + num
    return num

In [ ]:
def parse_address(moar, source):
    dict_list = []
    mes = []
    for row in moar.itertuples():
        addr_2 = clean_address_two(row.ADDRESS_2)
        address = f'{fix(row.ADDRESS_1)}{addr_2}, {fix(row.CITY)}, {fix(row.STATE)}'
        try:
            address_dict = usaddress.tag(address)[0]
        except usaddress.RepeatedLabelError as e:
            print(e.original_string)
            address_dict = error_handle(e.parsed_string)
            print('')
        address_dict['KEY'] = row.KEY
        address_dict['PHONE'] = row.PHONE
        address_dict['ZIPCODE'] = row.ZIPCODE
        address_dict['ZIP'] = fix_zipcode(row.ZIPCODE)
        dict_list.append(address_dict)
        mes.append(row.IQVIA_ME)
    parsed_df = pd.DataFrame(dict_list)
    parsed_df.dropna(how='all', axis=1, inplace=True)
    parsed_df.columns = [f'{c}_{source}' for c in parsed_df.columns.values]
    parsed_df['IQVIA_ME'] = mes
    return (parsed_df)

In [ ]:
for source_dict in source_list:
    source_dict['PARSED_DATA'] = parse_address(source_dict['DATA'], source_dict['SOURCE'])

In [ ]:
all_the_data = source_list[0]['PARSED_DATA'].merge(source_list[1]['PARSED_DATA'],on='IQVIA_ME',how='outer').merge(source_list[2]['PARSED_DATA'],on='IQVIA_ME',how='outer').merge(source_list[3]['PARSED_DATA'],on='IQVIA_ME',how='outer')

In [ ]:
all_the_data.columns

In [ ]:
all_the_data['ROW_KEY'] = list(range(0,len(all_the_data)))

In [ ]:
all_the_data = all_the_data.fillna('None')

In [ ]:
dict_list = []
for row in all_the_data.itertuples():
    count = 0
    phone_num = 'None'
    symph_phone = row.PHONE_SYMPHONY
    dhc_phone = row.PHONE_DHC
    iqvia_phone = row.PHONE_IQVIA
    data_phone = row.PHONE_GOV
    if symph_phone == dhc_phone and symph_phone != 'None':
        MATCHES = 'Symphony, DHC'
        phone_num = symph_phone
        if dhc_phone == iqvia_phone:
            count = 2
            MATCHES = 'Symphony, DHC, IQVia'
            if dhc_phone == data_phone:
                count = 3
                MATCHES = 'Symphony, DHC, IQVia, DataGov'
        elif dhc_phone == data_phone:
            count = 2
            MATCHES = 'Symphony, DHC, DataGov'
        else:
            count = 1       
    elif symph_phone == iqvia_phone and symph_phone != 'None':
        phone_num = symph_phone
        MATCHES = 'Symphony, IQVia'
        count = 1
        if symph_phone == data_phone:
            count = 2
            MATCHES = 'Symphony, IQVia, DataGov'
    elif symph_phone == data_phone and symph_phone != 'None':
        phone_num = symph_phone
        MATCHES = 'Symphony, DataGov'
        count = 1
        
    elif dhc_phone == iqvia_phone and dhc_phone != 'None':
        phone_num = dhc_phone
        MATCHES = 'DHC, IQVia'
        count = 1
        if dhc_phone == data_phone:
            MATCHES = 'DHC, IQVia, DataGov'
            count = 2
    elif dhc_phone == data_phone and dhc_phone != 'None':
        phone_num = dhc_phone
        MATCHES = 'DHC, DataGov'
        count = 1
    elif iqvia_phone == data_phone and iqvia_phone != 'None':
            phone_num = iqvia_phone
            MATCHES = 'IQVia, DataGov'
            count = 1
    if phone_num!='None':
        dicto = {
        'ROW_KEY': row.ROW_KEY,
        'IQVIA_ME': row.IQVIA_ME,
        'PHONE': phone_num,
        'MATCHED': count,
        'MATCHES': MATCHES
    }
        dict_list.append(dicto)

In [ ]:
phone_matches = pd.DataFrame(dict_list)

In [ ]:
phone_matches

In [ ]:
all_the_data

In [ ]:
for row in all_the_data[0:2].itertuples():
    print(row.LandmarkName_GOV + row.ZipCode_GOV)

In [ ]:
def count_matches(THIS):
    dict_list = []
    for row in THIS.itertuples():
        count = 0
        phone_num = 'None'
        symph_phone = row.AddressNumber_SYMPHONY + row.StreetName_SYMPHONY
        dhc_phone = row.AddressNumber_DHC + row.StreetName_DHC
        iqvia_phone = row.AddressNumber_IQVIA + row.StreetName_IQVIA
        data_phone = row.AddressNumber_GOV + row.StreetName_GOV
        if symph_phone == dhc_phone and symph_phone != 'NoneNone':
            MATCHES = 'Symphony, DHC'
            phone_num = symph_phone
            if dhc_phone == iqvia_phone:
                count = 2
                MATCHES = 'Symphony, DHC, IQVia'
                if dhc_phone == data_phone:
                    count = 3
                    MATCHES = 'Symphony, DHC, IQVia, DataGov'
            elif dhc_phone == data_phone:
                count = 2
                MATCHES = 'Symphony, DHC, DataGov'
            else:
                count = 1       
        elif symph_phone == iqvia_phone and symph_phone != 'NoneNone':
            phone_num = symph_phone
            MATCHES = 'Symphony, IQVia'
            count = 1
            if symph_phone == data_phone:
                count = 2
                MATCHES = 'Symphony, IQVia, DataGov'
        elif symph_phone == data_phone and symph_phone != 'NoneNone':
            phone_num = symph_phone
            MATCHES = 'Symphony, DataGov'
            count = 1

        elif dhc_phone == iqvia_phone and dhc_phone != 'NoneNone':
            phone_num = dhc_phone
            MATCHES = 'DHC, IQVia'
            count = 1
            if dhc_phone == data_phone:
                MATCHES = 'DHC, IQVia, DataGov'
                count = 2
        elif dhc_phone == data_phone and dhc_phone != 'NoneNone':
            phone_num = dhc_phone
            MATCHES = 'DHC, DataGov'
            count = 1
        elif iqvia_phone == data_phone and iqvia_phone != 'NoneNone':
            phone_num = iqvia_phone
            MATCHES = 'IQVia, DataGov'
            count = 1
        if phone_num!='None':
            dicto = {
            'ROW_KEY': row.ROW_KEY,
            'IQVIA_ME': row.IQVIA_ME,
            'ADDRESS': phone_num,
            'MATCHED': count,
            'ADDRESS_MATCHES': MATCHES
        }
            dict_list.append(dicto)
    return dict_list

In [ ]:
gah = count_matches(all_the_data)

In [ ]:
gah_match = pd.DataFrame(gah)

In [ ]:
gah_match

In [ ]:
fuzz.ratio('10820DRAKE', '10818DRAKE')

In [ ]:
def is_a_match(thing_1, thing_2):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>90:
        match = True
    else:
        match = False
    return(match)

def count_fuzzy_matches(THIS):
    dict_list = []
    for row in THIS.itertuples():
        count = 0
        phone_num = 'None'
        symph_phone = row.AddressNumber_SYMPHONY + row.StreetName_SYMPHONY
        dhc_phone = row.AddressNumber_DHC + row.StreetName_DHC
        iqvia_phone = row.AddressNumber_IQVIA + row.StreetName_IQVIA
        data_phone = row.AddressNumber_GOV + row.StreetName_GOV
        if is_a_match(symph_phone,dhc_phone) and symph_phone != 'NoneNone':
            MATCHES = 'Symphony, DHC'
            phone_num = symph_phone
            if is_a_match(dhc_phone,iqvia_phone):
                count = 2
                MATCHES = 'Symphony, DHC, IQVia'
                if is_a_match(dhc_phone,data_phone):
                    count = 3
                    MATCHES = 'Symphony, DHC, IQVia, DataGov'
            elif is_a_match(dhc_phone,data_phone):
                count = 2
                MATCHES = 'Symphony, DHC, DataGov'
            else:
                count = 1       
        elif is_a_match(symph_phone,iqvia_phone) and symph_phone != 'NoneNone':
            phone_num = symph_phone
            MATCHES = 'Symphony, IQVia'
            count = 1
            if is_a_match(symph_phone,data_phone):
                count = 2
                MATCHES = 'Symphony, IQVia, DataGov'
        elif is_a_match(symph_phone,data_phone) and symph_phone != 'NoneNone':
            phone_num = symph_phone
            MATCHES = 'Symphony, DataGov'
            count = 1

        elif is_a_match(dhc_phone,iqvia_phone) and dhc_phone != 'NoneNone':
            phone_num = iqvia_phone
            MATCHES = 'DHC, IQVia'
            count = 1
            if is_a_match(iqvia_phone,data_phone):
                MATCHES = 'DHC, IQVia, DataGov'
                count = 2
        elif is_a_match(dhc_phone,data_phone) and dhc_phone != 'NoneNone':
            phone_num = data_phone
            MATCHES = 'DHC, DataGov'
            count = 1
        elif is_a_match(iqvia_phone,data_phone) and iqvia_phone != 'NoneNone':
            phone_num = iqvia_phone
            MATCHES = 'IQVia, DataGov'
            count = 1
        if phone_num!='None':
            dicto = {
            'ROW_KEY': row.ROW_KEY,
            'IQVIA_ME': row.IQVIA_ME,
            'ADDRESS': phone_num,
            'MATCHED': count,
            'ADDRESS_MATCHES': MATCHES
        }
            dict_list.append(dicto)
    return dict_list

In [ ]:
bah = count_fuzzy_matches(all_the_data)

In [ ]:
add_match = pd.DataFrame(bah)

In [ ]:
bopbopbop = pd.merge(phone_matches, add_match, on=['ROW_KEY','IQVIA_ME'],suffixes=['_PHONE','_ADDRESS'], how='outer')

In [ ]:
iqvia[iqvia.IQVIA_ME.isin(bopbopbop.IQVIA_ME)==False].drop_duplicates('ME')

In [ ]:
len(bopbopbop)

In [ ]:
15750+3514

In [ ]:
REAL_DEAL = pd.merge(bopbopbop, all_the_data, on=['ROW_KEY', 'IQVIA_ME']).drop_duplicates(['IQVIA_ME','PHONE','ADDRESS'])

In [ ]:
all_the_data[all_the_data.ROW_KEY==0].iloc[0]

In [ ]:
KEY

In [ ]:
iqvia[iqvia.KEY=='23190IQVIA']['CITY'].values[0]

In [ ]:
count =0
for row in bopbopbop.itertuples():
    if row.MATCHES != row.ADDRESS_MATCHES:
        count+=1

In [ ]:
count

In [ ]:
for col in REAL_DEAL.columns:
    print(col)

In [ ]:
REAL_DEAL = REAL_DEAL.fillna('None')

In [ ]:
dictss = []
for row in REAL_DEAL.itertuples():
    if row.ADDRESS_MATCHES == 'None':
        print('Address mismatch')
        continue
    sources = row.ADDRESS_MATCHES
    address = row.ADDRESS
    phone = row.PHONE
    phone_source = 'Multiple'
    address_source = sources
    if "IQVia" in sources and row.StreetName_IQVIA in address:
        KEY = row.KEY_IQVIA
        state = row.StateName_IQVIA
        long_zipcode = row.ZIPCODE_IQVIA
        zipcode = row.ZIP_IQVIA
        city = iqvia[iqvia.KEY==KEY]['CITY'].values[0]
        address_1 = iqvia[iqvia.KEY==KEY]['ADDRESS_1'].values[0]
        address_2 = iqvia[iqvia.KEY==KEY]['ADDRESS_2'].values[0]
        if phone == 'None':
            phone = row.PHONE_IQVIA
            phone_source = 'IQVia'
        address_source = 'IQVia'
    elif "DataGov" in sources and row.StreetName_GOV in address:
        KEY = row.KEY_GOV
        state = row.StateName_GOV
        long_zipcode = row.ZIPCODE_GOV
        zipcode = row.ZIP_GOV
        city = gov[gov.KEY==KEY]['CITY'].values[0]
        address_1 = gov[gov.KEY==KEY]['ADDRESS_1'].values[0]
        address_2 = gov[gov.KEY==KEY]['ADDRESS_2'].values[0]
        if phone == 'None':
            phone = row.PHONE_GOV
            phone_source = 'DataGov'
        address_source = 'DataGov'
    elif "Symphony" in sources and row.StreetName_SYMPHONY in address:
        KEY = row.KEY_SYMPHONY
        state = row.StateName_SYMPHONY
        long_zipcode = row.ZIPCODE_SYMPHONY
        zipcode = row.ZIP_SYMPHONY
        city = symphony[symphony.KEY==KEY]['CITY'].values[0]
        address_1 = symphony[symphony.KEY==KEY]['ADDRESS_1'].values[0]
        address_2 = symphony[symphony.KEY==KEY]['ADDRESS_2'].values[0]
        if phone == 'None':
            phone = row.PHONE_SYMPHONY
            phone_source = 'Symphony'
        address_source = 'Symphony'
    NEW_DICT = {
        'ROW_KEY': row.ROW_KEY,
        'ME_IQVIA': row.IQVIA_ME,
        'ADDRESS_MATCHES': row.ADDRESS_MATCHES,
        'PHONE_MATCHES': row.MATCHES,
        'ADDRESS_1': address_1,
        'ADDRESS_2': address_2,
        'CITY': city,
        'STATE': state,
        'ZIPCODE_FULL': long_zipcode,
        'ZIPCODE': zipcode,
        'PHONE': phone,
        'PHONE_SOURCE': phone_source,
        'ADDRESS_SOURCE': address_source,
        'ADDRESS_KEY': address,
        'DATA_KEY': KEY
    }
    dictss.append(NEW_DICT)

In [ ]:
REAL_DEAL

In [ ]:
good_data = pd.DataFrame(dictss)

In [ ]:
extra_good_data = good_data[good_data.PHONE!='None']

In [ ]:
no_adds = REAL_DEAL[REAL_DEAL.IQVIA_ME.isin(good_data.ME_IQVIA)==False]

In [ ]:
no_adds

In [ ]:
phone_dictss = []
for row in no_adds.itertuples():
    sources = row.MATCHES
    phone = row.PHONE
    phone_source = 'Multiple'
    if "IQVia" in sources:
        KEY = row.KEY_IQVIA
        state = row.StateName_IQVIA
        long_zipcode = row.ZIPCODE_IQVIA
        zipcode = row.ZIP_IQVIA
        city = iqvia[iqvia.KEY==KEY]['CITY'].values[0]
        address_1 = iqvia[iqvia.KEY==KEY]['ADDRESS_1'].values[0]
        address_2 = iqvia[iqvia.KEY==KEY]['ADDRESS_2'].values[0]
        address_source = 'IQVia'
        address = row.AddressNumber_IQVIA+row.StreetName_IQVIA
    elif "DataGov" in sources:
        KEY = row.KEY_GOV
        state = row.StateName_GOV
        long_zipcode = row.ZIPCODE_GOV
        zipcode = row.ZIP_GOV
        city = gov[gov.KEY==KEY]['CITY'].values[0]
        address_1 = gov[gov.KEY==KEY]['ADDRESS_1'].values[0]
        address_2 = gov[gov.KEY==KEY]['ADDRESS_2'].values[0]
        address_source = 'DataGov'
        address = row.AddressNumber_GOV+row.StreetName_GOV
    elif "Symphony" in sources:
        KEY = row.KEY_SYMPHONY
        state = row.StateName_SYMPHONY
        long_zipcode = row.ZIPCODE_SYMPHONY
        zipcode = row.ZIP_SYMPHONY
        city = symphony[symphony.KEY==KEY]['CITY'].values[0]
        address_1 = symphony[symphony.KEY==KEY]['ADDRESS_1'].values[0]
        address_2 = symphony[symphony.KEY==KEY]['ADDRESS_2'].values[0]
        address_source = 'Symphony'
        address = row.AddressNumber_SYMPHONY+row.StreetName_SYMPHONY
    NEW_DICT = {
        'ROW_KEY': row.ROW_KEY,
        'ME_IQVIA': row.IQVIA_ME,
        'ADDRESS_MATCHES': row.ADDRESS_MATCHES,
        'PHONE_MATCHES': row.MATCHES,
        'ADDRESS_1': address_1,
        'ADDRESS_2': address_2,
        'CITY': city,
        'STATE': state,
        'ZIPCODE_FULL': long_zipcode,
        'ZIPCODE': zipcode,
        'PHONE': phone,
        'PHONE_SOURCE': phone_source,
        'ADDRESS_SOURCE': address_source,
        'ADDRESS_KEY': address,
        'DATA_KEY': KEY
    }
    phone_dictss.append(NEW_DICT)

In [ ]:
more_good_data = pd.concat([good_data, pd.DataFrame(phone_dictss)])

In [ ]:
more_good_data.drop_duplicates('ME_IQVIA').groupby('PHONE_MATCHES').count()[['ROW_KEY']]

In [ ]:
more_good_data.columns

In [ ]:
necessary_iqvia = iqvia[(iqvia.IQVIA_ME.isin(more_good_data.ME_IQVIA)==False)&(iqvia.PHONE!='None')].sort_values('AFFIL_RANK', ascending=False).drop_duplicates('ME', keep='first')

In [ ]:
['ROW_KEY', 'ME_IQVIA', 'ADDRESS_MATCHES', 'PHONE_MATCHES', 'ADDRESS_1',
       'ADDRESS_2', 'CITY', 'STATE', 'ZIPCODE_FULL', 'ZIPCODE', 'PHONE',
       'PHONE_SOURCE', 'ADDRESS_SOURCE', 'ADDRESS_KEY', 'DATA_KEY']
necessary_iqvia.columns
necessary_iqvia['ROW_KEY']='None'


In [ ]:
necessary_iqvia = pd.merge(necessary_iqvia, all_the_data, left_on='KEY',right_on='KEY_IQVIA').drop_duplicates('ME')

In [ ]:
necessary_iqvia['ADDRESS_MATCHES']='None'
necessary_iqvia['PHONE_MATCHES']='None'
necessary_iqvia['ZIPCODE_FULL']='None'
necessary_iqvia['ZIPCODE']='None'
necessary_iqvia['PHONE_SOURCE']='None'
necessary_iqvia['ADDRESS_SOURCE']='None'
necessary_iqvia['ADDRESS_KEY']='None'

In [ ]:
len(older_polos)

In [ ]:
len(iqvia.drop_duplicates('ME'))